# Graph CNN with Pytorch

## Import Modules

In [3]:
#以下这两个函数的作用是使用python3的语法，因为文章可能当初是用python2写的
#我把所有内容已经修改成python3的语法了
#from __future__ import division
#from __future__ import print_function

import time  #时间记录模块，python自带的
import numpy as np #numpy用于数据读取与存储
import scipy.sparse as sp #用于稀疏矩阵的压缩和转化，在数据中可能存在0的数量大于有值的数量的时候才需要使用
#可以大幅度提升效率，降低复杂度，无论是存储还是运算
import math  #数学工具，提供后文的一些数学计算

import torch  #标准torch环境模块，提供tensor类，将numpy数组转化成tensor，让计算机运行效率提升。类似tensorflow
import torch.nn as nn #nn是neural network的简称，用以简化搭建神经网络的难度，
#你在使用CNN和RNN这些经典层的时候可以直接调用nn.Conv2d,类似keras的使用方式
import torch.nn.functional as F  # 提供nn中的一些函数， 例如 F.relu，单列出来是因为functional敲起来太长了
import torch.optim as optim # 优化器模块，其实就是中间会调用到两次，固定模板，完全不需要理解，和keras一样


#因为pytorch的更新，下面两个基本上是不需要用了，因为variable和tensor整合了
#之前tensor是不能算梯度的，所以需要转化成variable，后来pytorch的人嫌麻烦，就改了源代码
from torch.nn.parameter import Parameter # 构建的网络的参数，此函数在新版pytorch中已经弃用
#新版直接A.parameters就完事了
from torch.nn.modules.module import Module # 自己构建的网络需要继承的模块，此函数在新版pytorch中已经弃用
#新版直接nn.Module放在类里就可以直接使用了

## Data Acquisition

这一段我的个人理解是：

load-data——获取数据和邻接矩阵
encode_onehot——标签独热码化，在load_data中调用。
normalize——将输入归一化，在load_data中调用，类似于batchnormalzation在输入层使用
sparse_mx_to_torch_sparse_tensor——用于将稀疏矩阵还原，在load_data中调用

load_data是用来将原始的数据按两部分做处理：1、构建邻接矩阵，2、将数据本身的标签和数据转化成两个矩阵
返回值是邻接矩阵， 特征（数据）， 标签，
encode_onehot是一个函数，用于将原本的标签转化成独热码。如果已经是独热码则无需这一步调用。
normalize是一个函数，输入任意矩阵，将矩阵每一行的总和计算出来，加和取倒数，如是稀疏矩阵，将其中无穷值转0。然后取倒数乘积运算。
sparse_mx_to_torch_sparse_tensor是给入稀疏矩阵，


todense是显示矩阵

### Main Dataloader

In [9]:
def load_data(path, dataset): #此处的path是一个‘//’的路径，dataset应为一个txt、pickle、mat等类型的矩阵
    #需要根据输入类型修改下面的数据获取函数
    """Load citation network dataset 
       读取数据，该数据应带有链接属性
    
    """
    
    print('Loading {} dataset...'.format(dataset)) #打印一下表明数据来源

    # —————————————————————————————————————————————————— #  

    # 读取样本id，特征和标签， 此处的np.genfromtxt是一个np内置的函数，用于从txt等列表、元组类型数据存储格式中获取数据
    idx_features_labels = np.genfromtxt(
        "{}{}.content".format(path, dataset), dtype=np.dtype(
            str))  # np.genfromtxt()生成 array： 文件数据的格式为id features    
    
    #此处原文获取的是文字数据，如果是使用时间序列则需改成如下:
    
    #from scipy.io import loadmat #通过scipy来读取，这一点和keras是一样的
    #matlab_data = loadmat("path/dataset.mat") 
    
    #获取到的数据的格式是一个带key的多元数组
    #可以用matlab_data.keys()来返回具体内容（通常是你自己命名的）
    
    # —————————————————————————————————————————————————— #
     #这一步的操作是为了提高内存的运行效率，因为过于稀疏的矩阵的特征转化成密集矩阵，在某种角度其实等同于
     #做了一个scale的缩减。
    #！但是由于你的输入不一定是特征，也不一定是稀疏矩阵，所以说未必需要使用。
    
    features = sp.csr_matrix(idx_features_labels[:, 1:-1],
                             dtype=np.float32)  # 提取样本的特征，并将其转换为csr矩阵
   
    #将label转化成独热码
    labels = encode_onehot(
        idx_features_labels[:, -1])  # 提取样本的标签，并将其转换为one-hot编码形式
    
    
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)  # 样本的id数组
    idx_map = {j: i for i, j in enumerate(idx)}  # 创建一个字典储存数据id

    ### 读取样本之间关系 ： 连边
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(
                         edges_unordered.shape)  # 无序边  map 成为有序

    # 构建邻接矩阵
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)  # 构建图的邻接矩阵
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(
        adj.T >
        adj)  # 矩阵进行对称化： 对于无向图，邻接矩阵是对称的。上一步得到的adj是按有向图构建的，转换成无向图的邻接矩阵需要扩充成对称矩阵。

    features = normalize(features)  # 对特征进行归一化处理
    adj = normalize(adj + sp.eye(adj.shape[0]))  #对邻接矩阵进行归一化处理

    idx_train = range(140)  # 训练集样本
    idx_val = range(200, 500)  # 验证集样本
    idx_test = range(500, 1500)  # 测试集样本

    # 从 numpy 转换为 torch
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

### Other support function

In [ ]:
def encode_onehot(labels): #就是生成label的独热码，你自己做出来的话这一个是可以省略的
    classes = set(labels)  # 注意因为 set ，所以每次生成的 y 的 onehot 值是不一样的
    classes_dict = {
        c: np.identity(len(classes))[i, :]
        for i, c in enumerate(classes)
    }  # np.identity() 创建一个单位对角阵， 单位矩阵的每一行对应一个one-hot向量
    labels_onehot = np.array(
        list(map(classes_dict.get, labels)),
        dtype=np.int32)  # map(function, iterable)： 对每个 label，应用 class_dict()
    return labels_onehot

In [ ]:
def normalize(mx): #归一化就是压缩到一定范围内，这里是为了降低scale
    """按行对矩阵进行归一化"""
    rowsum = np.array(mx.sum(1))  # 每行的值进行加和 ：x_sum =  (x_11 + x_12 + x_13 ...)
    r_inv = np.power(rowsum, -1).flatten()  # 加和的值取倒数  1/x_sum
    r_inv[np.isinf(r_inv)] = 0.  # 将结果中的无穷值转换为 0 ( x_sum可能为0 ，产生无穷值)
    r_mat_inv = sp.diags(r_inv)  # 将  1/x_sum 进行对角化
    mx = r_mat_inv.dot(mx)  # 初始矩阵和 1/x_sum 对角化矩阵进行乘积运算
    return mx

In [ ]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx): #这玩意对你没啥用，是一大堆0的数据才需要用的
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(
            np.int64))  # # 获得稀疏矩阵坐标 (2708, 1433)  --> (49216, 2)
    values = torch.from_numpy(sparse_mx.data)  # 相应位置的值 (49216, ) 即矩阵中的所有非零值
    shape = torch.Size(sparse_mx.shape)  # 稀疏矩阵的大小
    return torch.sparse.FloatTensor(indices, values, shape)

## Build Graph CNN Class

#### GCN有两种实现模式，一种是：设置GCN类，如下；另一种是在forward中加入点积即可
这里把GCN描述一遍，后面在forward那里我会给个加入点积位置的注释。

In [5]:
class GraphConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    #一个神经网络层、或者其他功能层的类在设置时通常涉及如下部分：
    #初始化：用于确认输入输出和一些基本的自带的属性，例如权重，偏置（通常偏置不需要你考虑，默认有就行了)
    #参数更新模式：用于更新内部的属性的输入接口，否则属性将无法自动更新，因为标准模块中名字是一样的，直接继承过来可以省去很多功夫
    #参数初始化模式：用于初始化parameter，因为parameter类中的初始化通常是0初始化
    #层内前向通路：forward，核心关键。在卷积层中这一部分是依次卷积，一重循环，在图卷积中则使用图卷积的前向过程
                   #其实就是变量怎么变换，想清楚了可以自定义任何计算层。
    #覆盖返回名称：通常init里面定义的内容，从外部直接用点就可以返回，如果想要好看一点，就用这个部分来设置。
    
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()  # 确保父类被正确的初始化了 #这里是为了能够继承标准层中的paramet这一个类，以简化后面的设置

        self.in_features = in_features #输入维度
        self.out_features = out_features #输出维度
        self.weight = Parameter(
            torch.FloatTensor(in_features, out_features)
        )  
        # 当Paramenters赋值给Module的属性的时候，他会自动的被加到 Module的 参数列表中 
        # 继承标准的parameter类，就可以更新了
        # 此处的torch.FloatTensor是一个将后面内容视作维度创建矩阵张量的函数。 parameter这个实现参数化

        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))  #这里可以看到如果输入一个维度，则生成一个1×N的矩阵可以用以加偏置
        else:
            self.register_parameter('bias', None)
            
        self.reset_parameters() # 进行参数初始化

    def reset_parameters(self):
        """参数初始化方式"""
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv) # 权重满足 #这里是使用uniform也就是高斯分布直接初始化
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv) #同理初始化偏置

    def forward(self, input_matrix, adj):
        #应该注意的是，在创建GNN的时候，使用
        
        support = torch.matmul(input_matrix, self.weight) # 将输入特征矩阵与权重参数矩阵相点积，注意这里有一个adj代表邻接矩阵
        #forward中需要给adj，因此在使用这一类的时候，会从类外环境调用adj这一个参数
        output = torch.spmm(adj, support) # 左乘标准化的邻接矩阵，邻接矩阵的存储时用的是稀疏矩阵 
        #这里如果之前不用稀疏矩阵这些东西，应该是改成:
        #output = torch.matmul(adj, support)
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        """输出类内部变量的名称"""
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

## Build GCN structure

In [25]:
#原本那个GCN的长度太短，这里这个是个长的。加入了一些

class GCN(nn.Module):
    def __init__(self, model, num_classes, in_channel, t=0, adj_file=None):
        super(GCNResnet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1,20,5),
            nn.ReLU(),
            nn.Linear(20, 10),
        ) #这个类似于keras的sequential创建，里面的内容可以更改.但是有一说一写习惯之后会觉得不是很好用
        
        self.pooling = nn.MaxPool2d(3, 2) #池化层使用模板
        self.dropout = dropout #可以定义dropout
        
        #正常使用的方式是：
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=8,kernel_size=(1, 3),stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1,padding=1)
        
        self.fc1 = nn.Linear(in_features=16*7*7,out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        

        self.gc1 = GraphConvolution(in_channel, 1024) #设置图卷积的init内容
        self.gc2 = GraphConvolution(1024, 2048)
        self.relu = nn.LeakyReLU(0.2) #这里你可以不写，直接在forward里面a = F.relu(a)就可以实现
        #这里是给你看看其他激活函数咋整


    def forward(self, x, adj):
        #第一种写法就是使用自己新建的类
        x = F.relu(self.features(x, adj)) # 第一层图卷积 
        x = F.dropout(x, self.dropout, training=self.training) 
        x = self.gc2(x, adj) # 第二层图卷积
        
        #如果是想要实现整体预测分类，你只需要保证最后的一层的输出节点数等于分类的类别数
        #然后最后用argmax+softmax来取值就好
        
        #第二种写法是直接用CNN，后面跟个mm：
        #x = torch.matmul(adj, x)
        #x = F.relu(self.conv1(input_size, output_size))
        #x = F.dropout(x, self.dropout, training=self.training) 
        #大概就类似这种，在卷积前×一个就好了
        
        #但是有一个问题，就是无论使用哪种图卷积的操作，应该保证维度要是对的，因为你后面跟个卷积，维度可能发生改变。
        #因为mm这类矩阵乘法需要用m*n与n*q的矩阵相乘得到结果，如果你得到的x的n*q发生了改变，adj和x的计算就会出问题。
        #我建议你看看别人这个地方是怎么处理的。
        
        return F.log_softmax(x, dim=1) # 计算 softmax + log 输出

## Instance

### pytorch的运行非常简单，分为以下部分：
1、超参数设置
2、实例化
3、优化器实例化
4、训练
5、测试画图

#### 实例化的超参数设置主要包含以下部分：
seed用来确认随机数，使得结果可复现：相当于是伪随机。
epochs迭代次数。
lr学习率。
weight_decay是一个正则化项。
hidden是隐层数量，可以在建模型的时候手动指定，也可以外部指定：外部指定的作用是可以做一个循环：for hidden in range 1，100.这样就可以尝试多种hidden的结果。
dropout你懂的。
fastmode = False 这一个选项是：dropout和BN在训练和测试的时候其实并不一样:测试的时候dropout是不运作的。默认false就好了。

In [7]:
seed = 42
epochs =200 # Number of epochs to train.
lr = 0.01 # Initial learning rate.
weight_decay = 5e-4 # Weight decay (L2 loss on parameters)
hidden = 16 # Number of hidden units.'
dropout = 0.5 # Dropout rate (1 - keep probability)
fastmode = False # val 时候是否和训练区分（dropout， BN）

### 数据与模型实例化

In [29]:
# 导入数据
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# 构建模型 #主要是设置内部的参数，其实如果你一开始就确定了参数，这里直接：
# model =GCN()就可以了
model = GCN(nfeat=features.shape[1],nhid=hidden,nclass=labels.max().item() + 1,dropout=dropout)

TypeError: __init__() missing 3 required positional arguments: 'model', 'num_classes', and 'in_channel'

### 优化器与损失函数实例化

In [27]:
# 创建优化器-因为pytorch当中优化器和反向传播全是pytorch来完成，
# 所以说需要在实例化的时候把这个设置好，后面一行代码解决
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) # 包含了权重正则化部分的 loss

#损失函数实例化也可以放到训练里面，其实无所谓在哪，能用就行
loss_train = F.nll_loss(
        output[idx_train], labels[idx_train]
    )  # ；计算损失与准确率；交叉熵loss， 因为模型计算包含 log， 原文这里使用 nll_loss（CrossEntropyLoss =Softmax+Log+NLLLoss）
#实际上如果你想用交叉熵：
loss_func = nn.CrossEntropyLoss() 
#然后再在训练中loss_func(y, label)就可以了

NameError: name 'model' is not defined

### 训练

pytorch中的网络模型训练非常非常模块化：
可以总结为：
设置自己是训练模式——把梯度置零——y = model（data，adj）——计算损失函数——一句话反向传播——一句话更新梯度

然后测试有好几种方式：你可以在训练一定次数后测试，也可以全整完之后测试，只需要把测试的位置改一改就行了

#### 简单的准确率计算函数

In [ ]:
def accuracy(output, labels):
    """计算准确率"""
    preds = output.max(1)[1].type_as(labels)  # 类型转换
    correct = preds.eq(labels).double()  # 是否相同， true， false
    correct = correct.sum()  # true false 加和
    return correct / len(labels)

#### 训练主体

In [30]:
def train(epoch): #这个内部的epoch其实没啥用，不用在意，只是为了迭代次数而已
    """标准 pytorch 神经网络流程"""
    t = time.time() # 这个是为了读时间，其实也不是必要的
    model.train()  # 先将model置为训练状态 #其实如果没有dropout，这一句是可以不要的
    
    output = model(features, adj) # 将输入送到模型得到输出结果 y = GCN(data, adj)
    loss_func = F.nll_loss(
        output[idx_train], labels[idx_train]
    )  #其实就是类似loss = CrossEntropyLoss(y, label)
    # ；计算损失与准确率；交叉熵loss， 因为模型计算包含 log， 这里使用 nll_loss（CrossEntropyLoss =Softmax+Log+NLLLoss）
    #如果前面定义了loss这里直接loss_func(y, label)就行
    
    #计算准确率
    acc_train = accuracy(output[idx_train], labels[idx_train])
    
    #！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！#
    #这三句就是任何神经网络中的梯度更新，任何模型照搬就行
    optimizer.zero_grad() # 梯度置0
    loss_func.backward() # 反向传播求梯度 loss_func是自己命名的名字
    optimizer.step() # 更新参数
    #！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！#
    
    #用来进行测试
    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval(
        )  #pytorch会自动把BN和DropOut固定住, dropout和batch normalization的操作在训练和测试的时候是不一样的
        #如果没有用dropout，就不用管，直接算就行
        output = model(features, adj)#y = GCN(data, adj)

    #类似上面的，这里的accuracy函数其实就是一个简单的计算函数
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    #打印
    print('Epoch: {:04d}'.format(epoch + 1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

#这一步其实就是之前测试的重复，只是最后再算一次而已
def test():
    model.eval() # 置为 evaluation 状态 
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:", "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

#### 实例化训练开始

In [ ]:
# 开始训练
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))